# parse 1 pdf

In [1]:
import os
from pypdf import PdfReader
import re
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

from markdownify import markdownify as md

In [2]:
import html2text
h = html2text.HTML2Text()


In [3]:
os.path.abspath("")

'c:\\!Work\\BDC_Chatbot'

In [4]:
dir_path = "./llm_docs/BDC_news/pdf" # working dir for jupyter notebook

current_dir = os.getcwd()
dir_path = os.path.join(current_dir, dir_path)
dir_path = os.path.normpath(dir_path)

files = os.listdir(dir_path)


In [5]:
file_name = files[0]
print(file_name)
file_path = os.path.join(dir_path, file_name)


reader = PdfReader(file_path)

2021 - Dec - BioData Catalyst_ December News.pdf


In [6]:
text = reader.get_page(0).extract_text()
# print(text)
print(text)

4/7/24, 5:04 PM BioData Catalyst: December News
https://mailchi.mp/f82e8e52e994/biodata-catalyst-december-news-14849820?e=[UNIQID] 1/6Email not displaying correctly? View this email in your browser
Welcome to the 
BioData Catalyst eNewsletter
December 2021
Announcements
Interest Groups Open
Starting in January 2022, three Interest Groups will be open to all users of BioData
Catalyst. These groups are:
Cross-Data Harmonization
CWL Development
GWAS/TOPMed
For more information on these groups and for details on how to join the meetings,
contact Amber at alvoght@renci.org. 
New Features and Release Highlights
The following are release highlights from the Q3 release. For a full list of items from
this release, see the release notes in our documentation.
BioData Catalyst Powered by Seven Bridges
Archive  les on AWS
Users on BioData Catalyst Powered by Seven Bridges can now select  les to move
from AWS S3 storage to AWS Glacier (archival storage). Moving  les to archival storage
can result in

In [7]:
# extract 1st url with 'https://mailchi.mp/'
def get_email_url(text):
    pattern = r'https://mailchi.mp/\S+'
    match = re.search(pattern, text)
    return match.group(0) if match else None

print(get_email_url(text))

https://mailchi.mp/f82e8e52e994/biodata-catalyst-december-news-14849820?e=[UNIQID]


In [8]:
# get html
url = get_email_url(text)
response = requests.get(url).text

soup = BeautifulSoup(response, 'html.parser')


In [9]:
# rm <div id="awesomewrap">
rm_div = soup.find('div', {'id': 'awesomewrap'})
rm_div.clear()
# rm <td id="templateFooter">
rm_td = soup.find('td', {'id': 'templateFooter'})
rm_td.clear()
# rm <table id="canspamBarWrapper">
rm_td = soup.find('table', {'id': 'canspamBarWrapper'})
rm_td.clear()
# rm <td id="templateHeader">
rm_td = soup.find('td', {'id': 'templateHeader'})
rm_td.clear()

In [10]:
response_md = md(str(soup))

In [11]:
print(h.handle(response_md))

BioData Catalyst: December News BioData Catalyst announcements, researcher spotlight, events, and more | | | | --- | | | | | Welcome to the BioData Catalyst eNewsletter **December 2021** | | --- | | | --- | --- | | | | | --- | | | --- | --- | | | Announcements | | --- | | | --- | --- | | | Interest Groups Open Starting in January 2022, three Interest Groups will be open to all users of BioData Catalyst. These groups are:* Cross\\-Data Harmonization * CWL Development * GWAS/TOPMed For more information on these groups and for details on how to join the meetings, contact Amber at [alvoght@renci.org](mailto:alvoght@renci.org). | | --- | | | --- | --- | | | New Features and Release Highlights The following are release highlights from the Q3 release. For a full list of items from this release, see the [release notes](https://bdcatalyst.gitbook.io/biodata-catalyst-documentation/written-documentation/release-notes/2021-10-04-biodata-catalyst-ecosystem-release-notes) in our [documentation](http

In [12]:
# extract links from markedown
def extract_links(text):
    pattern = r'\[([^\]]*)\]\(([^)]*)\)'
    # pattern = r'\[([^][]+)\](\(((?:[^()]+|(?2))+)\))'
    matches = re.findall(pattern, text)
    return matches

In [16]:
def get_BDC_email_text(dir_path):
    text_list = []
    link_list = []
    mail_urls = []
    files = os.listdir(dir_path)
    for file_name in tqdm(files):
        file_path = os.path.join(dir_path, file_name)
        reader = PdfReader(file_path)
        text = reader.get_page(0).extract_text()
        url = get_email_url(text)
        if url is None:
            print('No url found: ', file_name)
            response_md = ""
            mail_urls.append("")
        else:
            mail_urls.append(url)
            response = requests.get(url).text
            soup = BeautifulSoup(response, 'html.parser')
            rm_div = soup.find('div', {'id': 'awesomewrap'})
            rm_div.clear()
            rm_td = soup.find('td', {'id': 'templateFooter'})
            rm_td.clear()
            rm_td = soup.find('table', {'id': 'canspamBarWrapper'})
            rm_td.clear()
            rm_td = soup.find('td', {'id': 'templateHeader'})
            rm_td.clear()
            response_md = md(str(soup))
        
        text_list.append(h.handle(response_md))
        link_list.append(extract_links(response_md))
    
    return files, text_list, link_list, mail_urls

In [17]:
file_list, text_list, link_list, mail_urls = get_BDC_email_text(dir_path)

 95%|█████████▍| 18/19 [00:15<00:00,  1.30it/s]

No url found:  Campaign Overview _ Mailchimp.pdf


100%|██████████| 19/19 [00:15<00:00,  1.20it/s]


In [18]:
for i in range(len(file_list)):
    print(mail_urls[i])
    print(file_list[i])
    print(link_list[i]) 
    print()

https://mailchi.mp/f82e8e52e994/biodata-catalyst-december-news-14849820?e=[UNIQID]
2021 - Dec - BioData Catalyst_ December News.pdf
[('alvoght@renci.org', 'mailto:alvoght@renci.org'), ('release notes', 'https://bdcatalyst.gitbook.io/biodata-catalyst-documentation/written-documentation/release-notes/2021-10-04-biodata-catalyst-ecosystem-release-notes'), ('documentation', 'https://bdcatalyst.gitbook.io/biodata-catalyst-documentation/'), ('archival storage', 'https://sb-biodatacatalyst.readme.io/docs/file-archiving-overview'), ('Learn about', 'https://sb-biodatacatalyst.readme.io/docs/estimate-and-manage-your-cloud-costs'), ('rollout of PPWS', 'https://terra.bio/moving-to-a-project-per-workspace-model-for-improved-resource-management/'), ('set up and use GCP budget alerts', 'https://support.terra.bio/hc/en-us/articles/360057589931-How-to-set-up-and-use-GCP-budget-alerts'), ('Documentation', 'https://support.terra.bio/hc/en-us/articles/360037862771-How-much-did-a-workflow-analysis-cost-#h_

In [20]:
for i in range(len(file_list)):
    print(mail_urls[i])
    print(file_list[i])
    print(text_list[i])


https://mailchi.mp/f82e8e52e994/biodata-catalyst-december-news-14849820?e=[UNIQID]
2021 - Dec - BioData Catalyst_ December News.pdf
BioData Catalyst: December News BioData Catalyst announcements, researcher spotlight, events, and more | | | | --- | | | | | Welcome to the BioData Catalyst eNewsletter **December 2021** | | --- | | | --- | --- | | | | | --- | | | --- | --- | | | Announcements | | --- | | | --- | --- | | | Interest Groups Open Starting in January 2022, three Interest Groups will be open to all users of BioData Catalyst. These groups are:* Cross\\-Data Harmonization * CWL Development * GWAS/TOPMed For more information on these groups and for details on how to join the meetings, contact Amber at [alvoght@renci.org](mailto:alvoght@renci.org). | | --- | | | --- | --- | | | New Features and Release Highlights The following are release highlights from the Q3 release. For a full list of items from this release, see the [release notes](https://bdcatalyst.gitbook.io/biodata-catalys